In [3]:
import numpy as np
import numba
import copy
import matplotlib.pyplot as plt

sigma = 0.01                        # noise, called sigma_y in the cited paper
dim = 2                             # dimensionality of the space (v: R^d --> R)
m = 41

In [4]:
"""

NEEDED FUNCTIONS - true u

"""

N_trunc = 100

alpha = 1.1 #called s in the cited paper, as their alpha is 0, we omitted it here, as well as their sigma=1
def eigenvalues():
    ev = np.zeros((N_trunc,N_trunc))
    ev_root = np.zeros((N_trunc,N_trunc))
    for i in range(N_trunc):
        for j in range(N_trunc):
            ev[i,j] = 1/(np.pi**2*((i+1/2)**2+(j+1/2)**2))**alpha
    ev_root = ev**(1/2)
    return ev,ev_root
ev,ev_root = eigenvalues()

def xi_true():
    xi = np.zeros((N_trunc,N_trunc))
    for i in range(10):
        for j in range(10):
            xi[i,j] = ev_root[i+1,j+1]*np.sin((i+1/2)**2+(j+1/2)**2)
    return xi

In [5]:
"""

NEEDED FUNCTIONS - data

"""

# Define h where it is fixed (when x_2=0 or x_2=1).
h_first_row = np.zeros(m)
h_last_row = np.zeros(m)
for i in range(m): # Points on the bottom, where x_2=0 (h=x_1)
    h_first_row[i] = i/(m-1)
for i in range(m): # Points on top, where x_2=1 (h=1-x_1)
    h_last_row[i] = 1-i/(m-1)
    
"""
Initialise the finite difference scheme.
"""
def A_init(k):   
    A = np.zeros((m**2,m**2))
    for j in range(m-2): # central points
        for i in range(m-2):
            A[(i+1)+m*(j+1),(i+1)+m*(j+1)] = 2*k[(i+1)+m*(j+1)]+k[(i)+m*(j+1)]+k[(i+1)+m*(j)]
            A[(i+1)+m*(j+1),(i)+m*(j+1)] = -k[(i)+m*(j+1)]
            A[(i+1)+m*(j+1),(i+1)+m*(j)] = -k[(i+1)+m*(j)]
            A[(i+1)+m*(j+1),(i+2)+m*(j+1)] = -k[(i+1)+m*(j+1)]
            A[(i+1)+m*(j+1),(i+1)+m*(j+2)] = -k[(i+1)+m*(j+1)]
    i = m-1 # Points on the right, where x=6 (h_x=0)
    for j in range(m-2):
        A[i+m*(j+1),i+m*(j+1)] = k[i+m*(j+1)]+k[i+m*(j)]+k[i-1+m*(j+1)]
        A[i+m*(j+1),i+m*(j)] = -k[i+m*(j)]
        A[i+m*(j+1),i+m*(j+2)] = -k[i+m*(j+1)]
        A[i+m*(j+1),i-1+m*(j+1)] = -k[i-1+m*(j+1)]
    i = 0 # Points on the left, where x=0 (h_x=0)
    for j in range(m-2):
        A[i+m*(j+1),i+m*(j+1)] = 2*k[i+m*(j+1)]+k[i+m*(j)]
        A[i+m*(j+1),i+m*(j)] = -k[i+m*(j)]
        A[i+m*(j+1),(i+1)+m*(j+1)] = -k[i+m*(j+1)]
        A[i+m*(j+1),i+m*(j+2)] = -k[i+m*(j+1)]       
    # Corner points are already considered in the cases where x_2=0 and x_2=1.'''   
    A = 1/((6/(m-1))**2)*A   # Multiply by 1/Delta^2
    return A

"""
G maps a function u to the output of a PDE, h. It makes use of the finite 
difference approximation given by A.k and b_k. It's specific for the problem
setup.
"""
def G(k):    
    A_k = A_init(k)
    h = np.zeros(m**2)
    b = np.zeros(m**2)
    for i in range(m):
        b[i+m] -= A_k[i+m,i]*h_first_row[i]
        b[i+m*(m-2)] -= A_k[i+m*(m-2),i+m*(m-1)]*h_last_row[i]        
        
    # As the first and last column are fixed, we only consider a subset of the problem for the linalg solver.
    h_sub = np.zeros(m*(m-2))     
    A_sub = A_k[:,m:m*(m-1)]
    A_sub = A_sub[m:m*(m-1),:]
    h_sub = np.linalg.solve(A_sub,b[m:m*(m-1)])

    h[m:m*(m-1)] = h_sub
    h[0:m] = h_first_row
    h[m*(m-1):] = h_last_row    
    return h


obs_indices_list = [[9,0],[10,0],[11,0],[6,1],[14,1],[4,2],[16,2],[3,3],[17,3],[2,4],[18,4],[1,6],[19,6],[0,9],[20,9],[0,10],[10,10],[20,10],[0,11],[20,11],[1,14],[19,14],[2,16],[18,16],[3,17],[17,17],[4,18],[16,18],[6,19],[14,19],[9,20],[10,20],[11,20]] 
obs_indices = [] # for the 20 by 20 mesh
obs_indices_40 = [] # for the 40 by 40 mesh
for i in range(33):
    obs_indices.append(obs_indices_list[i][0]+21*obs_indices_list[i][1])
    obs_indices_40.append(2*obs_indices_list[i][0]+41*(obs_indices_list[i][1]*2))
np.save('GWF_obs_indices.npy',obs_indices)

In [16]:
"""
NEEDED FUNCTIONS
"""

''' Plotting a function exp(u) '''    
def func_plot(xi,name):
    x = np.arange(0,1,0.005)
    y = np.arange(0,1,0.005)
    X,Y = np.meshgrid(x,y)
    Z = np.zeros(X.shape)
    
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            x=np.zeros(dim)
            x[0]=X[i,j]
            x[1]=Y[i,j]
            Z[i,j] = np.exp(u(xi,x))

    fig, ax = plt.subplots()
    c = ax.pcolormesh(X, Y, Z, cmap='cool', vmin=Z.min(), vmax=Z.max())
    
    # set the limits of the plot to the limits of the data
    ax.axis([X.min(), X.max(), Y.min(), Y.max()])
    fig.colorbar(c, ax=ax)
    fig.savefig(name + '.pdf', dpi=300, bbox_inches='tight')
    plt.close(fig)
    
''' Plotting a function u '''    
def func_plot_log(xi,name):
    x = np.arange(0,1,0.005)
    y = np.arange(0,1,0.005)
    X,Y = np.meshgrid(x,y)
    Z = np.zeros(X.shape)
    
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            x=np.zeros(dim)
            x[0]=X[i,j]
            x[1]=Y[i,j]
            Z[i,j] = u(xi,x)

    fig, ax = plt.subplots()
    c = ax.pcolormesh(X, Y, Z, cmap='cool', vmin=-0.7, vmax=0.6)
    print('min(u) = ',Z.min())
    print('max(u) = ',Z.max())
    
    # set the limits of the plot to the limits of the data
    ax.axis([X.min(), X.max(), Y.min(), Y.max()])
    fig.colorbar(c, ax=ax)
    fig.savefig(name + '.pdf', dpi=300, bbox_inches='tight')
    plt.close(fig)
    
''' Plotting a function h '''    
def h_plot(h,name):
    x = np.linspace(0,1,m)
    y = np.linspace(0,1,m)
    X,Y = np.meshgrid(x,y)
    Z = np.zeros(X.shape)
    
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i,j] = h[i+m*j]

    fig, ax = plt.subplots()
    c = ax.pcolormesh(X, Y, Z, cmap='cool', vmin=Z.min(), vmax=Z.max())
    
    # set the limits of the plot to the limits of the data
    ax.axis([X.min(), X.max(), Y.min(), Y.max()])
    fig.colorbar(c, ax=ax)
    
    ''' add observation points '''
    obs_indices = np.load('GWF_obs_indices.npy')
    m_small=21
    x = []
    y = []
    for i in range(33):
        x.append((obs_indices[i]%m_small)/(m_small-1))
        y.append(((obs_indices[i]-obs_indices[i]%m_small)/m_small)/(m_small-1))
    ax.plot(x, y, "o")
    
    fig.savefig(name + '.pdf', dpi=300, bbox_inches='tight')
    plt.close(fig)

''' u(xi,x), which evaluates the function u with coefficients xi at x=(pos,speed) --- new, complicated, but fast version '''
@numba.njit()
def u(xi,x):
    u_sum = 0
    for i in range(N_trunc):
        for j in range(N_trunc):
            i_eval = np.pi*(i+1/2)*x[0]
            j_eval = np.pi*(j+1/2)*x[1]
            u_sum += xi[i,j]*np.cos(i_eval)*np.cos(j_eval)
    return 2*u_sum

In [17]:
"""

Create data - as in the GWF example

"""

np.random.seed(69)

xi = xi_true()
func_plot(xi,'figs/GWF_exp_u_true')
func_plot_log(xi,'figs/GWF_u_true')

k = np.zeros(m**2)
for j in range(m):
    for i in range(m):
        k[i+m*j] = np.exp(u(xi,(i/(m-1),j/(m-1))))
h = G(k)
h_plot(h,'figs/GWF_h')

y = h[obs_indices_40]+sigma*np.random.normal(size=33)

np.save('GWF_data.npy',y)

min(u) =  -0.6533510593629042
max(u) =  0.5432728002947831


In [ ]:
"""

Create data - no forward model, just observe u directly

"""

m = 21

np.random.seed(42)

xi = xi_true()

k = np.zeros(m**2)
for j in range(m):
    for i in range(m):
        k[i+m*j] = u(xi,(i/(m-1),j/(m-1)))

y = k+sigma*np.random.normal(size=m**2)

np.save('GWF_data_all_u.npy',y)